In [1]:
from biogeodataframe import BioGeoDataFrame
from osgeo import gdal
import geopandas as gpd
from rioxarray.merge import merge_arrays
from geocube.api.core import make_geocube
import numpy as np

In [2]:
# Set the CRS to BC Albers
CRS = 'EPSG:3005'
GEOCUBE_RES = 100
N_SAMPLES = 5000

In [3]:
# Read in species occurrence data as a geodataframe and remove non-georeferenced rows
species_tmp = gpd.read_file('../data/black_bear_occurrences.csv')
species_tmp = species_tmp[(species_tmp['decimalLatitude'] != '') & (species_tmp['decimalLongitude'] != '')]

In [4]:
# Convert the geopandas to a BioGeoDataFrame, giving access to useful methods
N = np.nanmin((N_SAMPLES, species_tmp.shape[0]))
species_tmp = species_tmp.sample(N)

species = BioGeoDataFrame(species_tmp)
species = species.set_geometry(gpd.points_from_xy(
        species['decimalLongitude'], species['decimalLatitude'])).set_crs(4326)
species = species.to_crs(CRS)

/opt/homebrew/anaconda3/envs/tensorflow/lib/python3.9/site-packages/geopandas/geodataframe.py:206: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  super().__setattr__(attr, val)


In [5]:
# Load in biogeoclimatic zones and reproject to desired CRS
# Use only the ZONE and geometry fields, the former of which is what we will predict species' distributions with
bec_tmp = gpd.read_file('../data/bec').to_crs(CRS)
bec_tmp = bec_tmp[['ZONE', 'SUBZONE', 'geometry']]

In [6]:
# Categorical variables must be made numeric to be transformed into a raster, so must convert numbers back to strings
# To do this, create list of all strings
bec_zones = bec_tmp.ZONE.drop_duplicates().values.tolist()
bec_subzones = bec_tmp.SUBZONE.drop_duplicates().values.tolist()
categorical_enums = {"ZONE": bec_zones, "SUBZONE": bec_subzones}

In [7]:
# Convert bec geodataframe to rioxarray raster
# Resolution is in the units of target CRS
bec = make_geocube(vector_data = bec_tmp, resolution=(GEOCUBE_RES, -GEOCUBE_RES), categorical_enums=categorical_enums)

In [8]:
# print(np.unique(bec['ZONE']))
# print(np.unique(bec['ZONE'].astype(int)))

In [9]:
# Convert numeric back to categorical string
######################################### DO NOT DELETE ######################################### 
# zone_string = bec['ZONE_categories'][bec['ZONE'].astype(int)].drop('ZONE_categories')
# bec['ZONE'] = zone_string

In [10]:
# Create pseudo-absences
pres_abs = species.add_pseudo_absences(amount=species.shape[0], region_poly=bec_tmp)

1360 pseudo-absence points remaining.
748 pseudo-absence points remaining.
405 pseudo-absence points remaining.
222 pseudo-absence points remaining.
124 pseudo-absence points remaining.
68 pseudo-absence points remaining.
34 pseudo-absence points remaining.
19 pseudo-absence points remaining.
12 pseudo-absence points remaining.
9 pseudo-absence points remaining.
4 pseudo-absence points remaining.
2 pseudo-absence points remaining.
1 pseudo-absence points remaining.


/opt/homebrew/anaconda3/envs/tensorflow/lib/python3.9/site-packages/geopandas/geodataframe.py:206: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  super().__setattr__(attr, val)


In [11]:
BUFFER_DISTANCE = bec.rio.resolution()[1] * 31.5 # in units of CRS

In [12]:
# Given a list of raster tiles, find which ones intersect the species occurrence points and are therefore required
# Using a single raster, bec, for simplicity
rasters = pres_abs.list_rasters(BUFFER_DISTANCE, [bec])

In [13]:
# Load the list of raster tiles into memory
# Would load the rasters here, but bec is already loaded for simplicity. Something like:
# rasters = [rioxarray.open_rasterio(x) for x in raster]
# merged_raster = merge_arrays(rasters)
merged_raster = bec

In [14]:
# Buffer each point so it intersects adjacent raster cells
# pres_abs['buffered_geometry'] = pres_abs['geometry'].buffer(BUFFER_DISTANCE, cap_style=3)

In [15]:
# For each occurrence point, build a 3D tensor
vals = pres_abs.extract_values(raster=merged_raster, distance=BUFFER_DISTANCE)
vals = np.concatenate(vals)

[array([[8, 8, 8, ..., 8, 8, 8],
       [8, 8, 8, ..., 8, 8, 8],
       [8, 8, 8, ..., 8, 8, 8],
       ...,
       [8, 8, 8, ..., 8, 8, 8],
       [8, 8, 8, ..., 8, 8, 8],
       [8, 8, 8, ..., 8, 8, 8]], dtype=int16), array([[4, 4, 4, ..., 4, 4, 4],
       [4, 4, 4, ..., 4, 4, 4],
       [4, 4, 4, ..., 4, 4, 4],
       ...,
       [4, 4, 4, ..., 4, 4, 4],
       [4, 4, 4, ..., 4, 4, 4],
       [4, 4, 4, ..., 4, 4, 4]], dtype=int16)]
[array([[2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2],
       ...,
       [2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2]], dtype=int16), array([[17, 17, 17, ..., 17, 17, 17],
       [17, 17, 17, ..., 17, 17, 17],
       [17, 17, 17, ..., 17, 17, 17],
       ...,
       [17, 17, 17, ..., 17, 17, 17],
       [17, 17, 17, ..., 17, 17, 17],
       [17, 17, 17, ..., 17, 17, 17]], dtype=int16)]
[array([[15, 15, 15, ..., 15, 15, 15],
       [15, 15, 15, ..., 15, 15, 15],
       [1

In [16]:
# Import required packages
import tensorflow as tf
import keras
from keras import layers
import pandas as pd
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [17]:
# listt = []
# listt.append(np.array((((1, 1), (2, 2)), ((1, 1), (2, 2)))))
# arr2 = np.array((((2, 2), (4, 4)), ((4, 4), (4, 4))))

# # np.stack((listt, arr2)).shape
# np.stack((listt))
# # arr2

In [18]:
# # [x['presence'] for x in vals if None not in x['arr'][0] and 'nodata' not in x['arr'][1]].__len__()
# l = [
#     x["arr"]
#     for x in vals
# ]
# vals
# # [x.shape for x in l]

In [19]:
# np.array((((2, 2), (3, 3)), ((2, 2), (3, 3)), ((2, 2), (3, 3))))
# np.zeros((255,255,3))
# x_train[0].transpose().shape

In [20]:
x_train = np.stack(
    [x["arr"].transpose() for x in vals]
)  # if None not in x['arr'] is not None and 'nodata' not in x['arr']], axis=0)
y_train = np.stack(
    [x["presence"] for x in vals]
)  # if x['arr'] is not None and 'nodata' not in x['arr']])

In [24]:
# model = tf.keras.models.Sequential([
#   # tf.keras.layers.Input(shape=(1,)),
#   tf.keras.layers.Flatten(),
#   tf.keras.layers.Dense(4, activation='relu'),
#   tf.keras.layers.Dense(4, activation='relu'),
#   tf.keras.layers.Dense(2, activation='softmax')
# ])

model = tf.keras.models.Sequential()
model.add(Conv2D(32, (2, 2), input_shape=(64, 64, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) # downsample each dimension by a factor of 2

model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(2)) # This should be the number of layers
model.add(Activation('softmax'))
# len(model.weights)

In [25]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
m = model.fit(x_train, y_train, batch_size=128, epochs=100)

Epoch 1/100
40/40 [==============================] - 1s 19ms/step - loss: 8.2801 - accuracy: 0.4940
Epoch 2/100
40/40 [==============================] - 1s 17ms/step - loss: 1.0819 - accuracy: 0.5253
Epoch 3/100
40/40 [==============================] - 1s 18ms/step - loss: 0.8958 - accuracy: 0.5474
Epoch 4/100
40/40 [==============================] - 1s 18ms/step - loss: 0.7822 - accuracy: 0.5717
Epoch 5/100
40/40 [==============================] - 1s 18ms/step - loss: 1.2447 - accuracy: 0.5783
Epoch 6/100
40/40 [==============================] - 1s 17ms/step - loss: 2.1951 - accuracy: 0.5556
Epoch 7/100
40/40 [==============================] - 1s 17ms/step - loss: 1.7535 - accuracy: 0.5927
Epoch 8/100
40/40 [==============================] - 1s 17ms/step - loss: 1.5956 - accuracy: 0.6010
Epoch 9/100
40/40 [==============================] - 1s 17ms/step - loss: 1.2275 - accuracy: 0.6092
Epoch 10/100
40/40 [==============================] - 1s 17ms/step - loss: 1.1142 - accuracy: 0.6315